# Import

In [1]:
!pip install torchsummaryX

In [2]:
import torch
import numpy as np
import torch.nn as nn
from torchsummaryX import summary

# Constants

In [3]:
num_quantiles = 10 # H_in
look_back_window = 5 # L
hidden_size = 128
n = 50

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model

In [4]:
x = torch.randn((look_back_window, 1, num_quantiles))

In [5]:
class Network(nn.Module):

    def __init__(self, input_size, output_size, hidden_size): # You can add any extra arguments as you wish

        super(Network, self).__init__()
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, bidirectional=False)

        self.classification = nn.Linear(in_features=hidden_size, out_features=output_size)

    def forward(self, x):

        output, (h_n, c_n) = self.lstm(x)
        # output: (L, 1 * H_out)

        out = self.classification(output)

        return out

model = Network(num_quantiles, num_quantiles, hidden_size).to(device)
summary(model, x.to(device)) 

                 Kernel Shape Output Shape  Params  Mult-Adds
Layer                                                        
0_lstm                      -  [5, 1, 128]   71680      70656
1_classification    [128, 10]   [5, 1, 10]    1290       1280
-------------------------------------------------------------
                      Totals
Total params           72970
Trainable params       72970
Non-trainable params       0
Mult-Adds              71936


/usr/local/lib/python3.7/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_lstm,-,"[5, 1, 128]",71680,70656
1_classification,"[128, 10]","[5, 1, 10]",1290,1280


In [9]:
x_train = torch.randn(n, look_back_window, 1, num_quantiles) # n
y_train = torch.randn(n, 1, num_quantiles)

# Model Parameters

In [10]:
epochs = 50
lr = 0.001
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Model Training

In [17]:
for epoch in range(epochs):
    arr = np.arange(len(x_train))
    np.random.shuffle(arr)
    for ind in arr:
        seq = x_train[ind]
        labels = y_train[ind]
        optimizer.zero_grad()

        y_pred = model(seq)

        single_loss = loss_function(y_pred[-1, :, :], labels)
        
        single_loss.backward()
        optimizer.step()

    print(f"[LSTM FIT]epoch: {epoch + 1:3} loss: {single_loss.item():10.8f}")

[LSTM FIT]epoch:   1 loss: 0.00138746
[LSTM FIT]epoch:   2 loss: 0.00189571
[LSTM FIT]epoch:   3 loss: 0.00063254
[LSTM FIT]epoch:   4 loss: 0.00513293
[LSTM FIT]epoch:   5 loss: 0.00193271
[LSTM FIT]epoch:   6 loss: 0.00686951
[LSTM FIT]epoch:   7 loss: 0.00086578
[LSTM FIT]epoch:   8 loss: 0.00623362
[LSTM FIT]epoch:   9 loss: 0.00367014
[LSTM FIT]epoch:  10 loss: 0.00237094
[LSTM FIT]epoch:  11 loss: 0.00544881
[LSTM FIT]epoch:  12 loss: 0.00402756
[LSTM FIT]epoch:  13 loss: 0.00496324
[LSTM FIT]epoch:  14 loss: 0.00188385
[LSTM FIT]epoch:  15 loss: 0.00219478
[LSTM FIT]epoch:  16 loss: 0.00066603
[LSTM FIT]epoch:  17 loss: 0.00073235
[LSTM FIT]epoch:  18 loss: 0.00089776
[LSTM FIT]epoch:  19 loss: 0.00103045
[LSTM FIT]epoch:  20 loss: 0.00056660
[LSTM FIT]epoch:  21 loss: 0.00164224
[LSTM FIT]epoch:  22 loss: 0.00133327
[LSTM FIT]epoch:  23 loss: 0.00154545
[LSTM FIT]epoch:  24 loss: 0.00051522
[LSTM FIT]epoch:  25 loss: 0.00065057
[LSTM FIT]epoch:  26 loss: 0.00046732
[LSTM FIT]ep